In [1]:
from ChEEssentials import *
import ChEEssentials as ChE

In [2]:
import numpy as np
import waterproperties as water
from scipy.optimize import fsolve, curve_fit

In [3]:
print(water.hvp(350))
print(water.hvp(400))

41770.72809521174
39347.6051451064


In [16]:
L = 0.3556 * m #m
BS = 4*inch
g = 9.8 * m /s**2 #gravity 
Di = 0.206*inch  #diameter of pipe
Do = 0.25*inch
n = 56 #check this. num of tube 
k = 14 *W/m/K
Tsat = 100+273
Ai = np.pi * Di*L
Ao = np.pi * Do*L
P_a = 84.83*kPa #ambient
print(Ai,Ao)

0.0058454 m^2  0.0070939 m^2 


In [17]:
def Tst(Ps):
    Ps = Ps*psi + P_a
    Ps = Ps.getNum()
    return water.tsat(Ps) - 273

Tsat = Tst(10) + 273

def Q(f):
    f = f*gal/min
    return f/n

def P(Ts,Tsat):
    BS = 4*inch
    T = (Tsat + Ts)/2
    hvap = water.hvp(T) * J / mol
    Ja = water.lcp(T)*J/mol/K * (Tsat - Ts)*K/hvap
    hvapp = hvap*(1 + 0.68*Ja)
    return (water.ltc(T)*W/m/K)*BS*(Tsat - Ts)*K/(water.lvs(T)*Pa*s)/hvapp/((water.nu(T)*m**2/s)**2/g)**(1/3)*ChE.MWU("H2O")

def ho(Ts,Tsat):
    Tsat += 273
    Ts += 273
    T = (Tsat + Ts)/2
    Nuo = 1/P(Ts,Tsat)*(0.68*P(Ts,Tsat) + 0.89)**(0.82)
    h_o = Nuo*(water.ltc(T)*W/m/K)/((water.nu(T)*m**2/s)**2/g)**(1/3) #W/m^2/K
    return h_o

def q(Ti,To,f):
    Ti += 273
    To += 273
    T = (Ti+To)/2
    return water.ldn(T)*kg/m**3*Q(f)*water.lcp(T)*J/mol/K*(To-Ti)*K/ChE.MWU("H2O")


def hi(Ti,To,f):
    Ti = Ti+273
    To = To+273
    Tt = (Ti+To)/2
    Re = 4*Q(f)/np.pi/Di/(water.nu(Tt)*m**2/s)
    ff = (0.790*ChE.log(Re) - 1.64)**-2
    Pr = water.pr(Tt)
    Nud = (ff/8*(Re - 1000)*Pr)/((1 + 12.7*(ff/8)**0.5)*(Pr**(2/3)-1))
    h_i = Nud*water.ltc(Tt)*W/m/K/Di #W/m^2/K
    return h_i


print(hi(52, 63, 20))
print(P(20,30))

3.9568e+05 kg / K s^3.0 
-8.016e-85 


In [18]:
def getHo(Tsat, Tci, Tco, f):
    def getTs(Ts):
        Ts = Ts[0]
        A = q(Tci, Tco, f)
        B = ho(Ts, Tsat)*Ao*(Tsat-Ts)*K
        return (A-B).getNum()
    Ts = fsolve(getTs, (Tsat+Tco)/2)[0]
    return ho(Ts, Tsat)

print(getHo(100, 50, 60, 20))

1.1442e+04 kg / K s^3.0 


In [27]:
def ΔTlm(Thi,Tho,Tci,Tco):
    return ((Thi-Tco)-(Tho-Tci))/((np.log((Thi-Tco)/(Tho-Tci))))

print(ΔTlm(112,112,52.49,63.47))

def UA(f,Thi,Tho,Tci,Tco):
    return q(Tci,Tco,f)/ΔTlm(Thi,Tho,Tci,Tco)/K
    
def Rf(f,Thi,Tho,Tci,Tco):
    a = 1/UA(f,Thi,Tho,Tci,Tco)
    b = 1/hi(Tci,Tco,f)/Ai
    c = ChE.log(Do/Di)/2/np.pi/k/L
    d = 1/getHo(Thi, Tci, Tco, f)/Ao
    return (a - b - c - d)*Ai #K/W


print(Rf(20.00,112,112,52.49,63.47))
print(1/UA(20.00,112,112,52.49,63.47))
print(1/hi(52.49,63.47,20)/Ai)
print(ChE.log(Do/Di)/2/np.pi/k/L)
print(1/getHo(112, 52.49, 63.47, 20)/Ao)

53.833503852343995
0.0002005 K s^3.0 / kg 
0.052869 K s^3.0 / kg m^2.0 
0.00042763 K s^3.0 / kg m^2 
0.0061887 K s^3.0 / kg m^2.0 
0.011952 K s^3.0 / kg m^2 
